In [1]:
from DeepLearning.PPO import MaskablePPO
from DeepLearning.Thesis.Environments.ObservationTesting import ObsTestingEnv
from DeepLearning.GetActionMask import getActionMask
from DeepLearning.Thesis.Observations.get_observation import getObservation, lowerBound, upperBound
import os

env = ObsTestingEnv(lowerBounds=lowerBound, upperBounds=upperBound, getObservationFunction=getObservation)
actionMask = getActionMask
observation = getObservation

saveName = "final_observation"
savePath = f"DeepLearning/Thesis/Observations/Models/{saveName}"

model = MaskablePPO("MlpPolicy", env, verbose=1, getActionMask=actionMask, getObservation=observation, savePath=savePath) #, tensorboard_log="./tensorboard_logs_thesis/")
model.learn(total_timesteps=10_000)# , tb_log_name=saveName)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 94.5     |
|    ep_rew_mean     | 6.67     |
| time/              |          |
|    fps             | 1334     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 94.6        |
|    ep_rew_mean          | 6.05        |
| time/                   |             |
|    fps                  | 937         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016774988 |
|    clip_fraction        | 0.192       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.92       |
|    explained_variance   | -0.00417    |
|    learning_rate        | 0.

/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:283: UserWarning: Path 'DeepLearning/Thesis/Observations/Models/final_observation' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [2]:
"""
Running Agent simulations
"""
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentMCTS import AgentMCTS
from Agents.AgentUCT import AgentUCT
from Agents.AgentModel import AgentMultiModel, AgentModel
from Game.CatanGame import *
from CatanSimulator import CreateGame
from DeepLearning.PPO import MaskablePPO
from Game.CatanPlayer import PlayerStatsTracker
from tabulate import tabulate
from DeepLearning.Stats import headers
import dill as pickle
from CatanData.GameStateViewer import SaveGameStateImage, DisplayImage
import math
from DeepLearning.Environments.CatanEnv import CatanEnv, SelfPlayDistribution, CatanTradingEnv, SelfPlayDistTradingEnv

from DeepLearning.Thesis.Observations.nodes_v_hexes import getNodeInHexObs, nodeInHexLowerBound, nodeInHexUpperBound

winner = [0,0,0,0]
player0Stats = PlayerStatsTracker()
Player0LosingStats = PlayerStatsTracker()
player1Stats = PlayerStatsTracker()

# bestModel = MaskablePPO.load("DeepLearning/Models/Full/Rewards_SetupVpActionDenseTrades_8M.zip")

testModel = MaskablePPO.load("DeepLearning/Thesis/Observations/Models/final_observation/model_10240.zip")
# testModel2 = MaskablePPO.load("DeepLearning/Thesis/Observations/Models/observation_hex/Final.zip")

players = [ AgentModel("P0", 0, recordStats=True, playerTrading=False, model=testModel),
            # AgentModel("P1", 1, recordStats=True, playerTrading=False, model=testModel2),
            AgentRandom2("P1", 1, recordStats=True, playerTrading=False),
            AgentRandom2("P2", 2, recordStats=True, playerTrading=False),
            AgentRandom2("P3", 3, recordStats=True, playerTrading=False),]


COLLECT_STATS = True
for episode in range(1):
    game = CreateGame(players)
    game = pickle.loads(pickle.dumps(game, -1))
    numTurns = 0
    while True:
        currPlayer = game.gameState.players[game.gameState.currPlayer]

        agentAction = currPlayer.DoMove(game)
        agentAction.ApplyAction(game.gameState)

        if currPlayer.seatNumber == 0 and agentAction.type == 'EndTurn':
        #     DisplayImage(game.gameState, agentAction)
        #     time.sleep(1)
            numTurns += 1

        if game.gameState.currState == "OVER":
            # DisplayImage(game.gameState, agentAction)
            break
    
    # print("Winner: ", game.gameState.winner)
    winner[game.gameState.winner] += 1
    lost = game.gameState.winner != 0

    # Stats
    if COLLECT_STATS:
        game.gameState.players[0].generatePlayerStats()
        game.gameState.players[1].generatePlayerStats()

        player0Stats += game.gameState.players[0].stats
        player1Stats += game.gameState.players[1].stats
        if lost:
            Player0LosingStats += game.gameState.players[0].stats

# Collect stats
if COLLECT_STATS:
    player0Stats.getAverages()
    Player0LosingStats.getAverages()
    player1Stats.getAverages()
    player0Data = player0Stats.getList()
    player0LosingData = Player0LosingStats.getList()
    player1Data = player1Stats.getList()

    p_hat0 = winner[0] / sum(winner)
    p_hat1 = winner[1] / sum(winner)
    margin_error0 = round(100*(1.96 * math.sqrt((p_hat0 * (1 - p_hat0)) / sum(winner))), 2)
    margin_error1 = round(100*(1.96 * math.sqrt((p_hat1 * (1 - p_hat1)) / sum(winner))), 2)
    player0Data.insert(0, margin_error0)
    player0LosingData.insert(0, -1)
    player1Data.insert(0, margin_error1)
    player0Data.insert(0, winner[0]/sum(winner))
    player0LosingData.insert(0, -1)
    player1Data.insert(0, winner[1]/sum(winner))
    player0Data.insert(0, "Player0")
    player0LosingData.insert(0, "Player0LossesStats")
    player1Data.insert(0, "Player1")

    table = tabulate([player0Data, player0LosingData, player1Data], headers=headers, tablefmt='simple')
    print(table)

print(f"\nNum turns: {numTurns}")

print("\n\nWinnings: ", winner)


# Brick, ore, wool, wheat, wood

AgentName             WinRate    MarginError    numTurns    victoryPoints    numRoadsBuilt    devCardsBought  usedDevCards                 settlementsBuilt    citiesBuilt    devCardVP    largestArmy    longestRoad  resourcesReceived                 totalResourcesReceivedPerTurn    totalResourcesDiscarded    totalResourcesStolen  resourcesFromDevCard         totalResourcesFromDevCard  resourcesFromBankTrade        finalResourceProduction      finalTradeRates            setupResourceProduction      totalSetupResourceProduction  setupTradeRates              setupResourceDiversity    turnsForFirstSettlement    noSettlementsBuilt    turnsForFirstCity    noCitysBuilt    numRoadsFor1stSettlement    totalResourcesFromBankTrade    goodSettlementBankTrades    badSettlementBankTrades    goodCityBankTrades    badCityBankTrades    goodRoadBankTrades    badRoadBankTrades    goodDevCardBankTrades    badDevCardBankTrades    neutralBankTrades    acceptedTrades    rejectedTrades    goodSettlementAccepte

In [7]:
# import pandas as pd

# # Save to csv
# fileName = f'singeOpponentObs_v_defaultOpponentObs.csv'
# df = pd.DataFrame([player0Data, player0LosingData, player1Data], columns=headers)
# df.to_csv(f'DeepLearning/Thesis/Observations/Data/{fileName}', index=False)